In [ ]:
import yfinance as yf 
import pandas as pd

# The stock tickers for your 23 companies
tickers = ["AMR","APA","AR","ARCH","ARLP","BTU","CEIX","EXE","CLR","CNX","COP","CVX","DVN","EOG","EQT","HES","MRO","MUR","OXY","PXD","SM","EXE","VST"]  # Replace with your 23 tickers
start_year = 1993
end_year = 2023

# Create an empty DataFrame to store the data
all_data = []

for ticker in tickers:
    print(f"📊 Retrieving data for {ticker}...")

    stock = yf.Ticker(ticker)

    # Get stock price data
    data = yf.download(ticker, start=f"{start_year}-01-01", end=f"{end_year}-12-31")

    if data.empty:
        print(f"⚠️ Unable to retrieve data for {ticker}, skipping...")
        continue

    # Select the Close price
    if 'Close' in data.columns:
        price_col = 'Close'
    elif 'Adj Close' in data.columns:
        price_col = 'Adj Close'
    else:
        print(f"⚠️ {ticker} does not have Close or Adj Close data, skipping...")
        continue

    # Resample data to annual frequency, calculating the close price, average, high, and low
    yearly_data = data[[price_col]].resample('Y').agg(['last', 'mean', 'max', 'min'])
    yearly_data.columns = ["Year_End_Close", "Year_Avg_Close", "Year_High_Close", "Year_Low_Close"]
    yearly_data["Yearly_Return"] = yearly_data["Year_End_Close"].pct_change()

    # Add Ticker and Year columns
    yearly_data["Ticker"] = ticker
    yearly_data["Year"] = yearly_data.index.year

    # **Calculate Market Cap and PE Ratio**
    try:
        info = stock.info
        shares_outstanding = info.get("sharesOutstanding", None)  # Total number of shares
        if shares_outstanding:
            yearly_data["Market_Cap"] = yearly_data["Year_End_Close"] * shares_outstanding  # Calculate historical market cap
        else:
            yearly_data["Market_Cap"] = None

        # Calculate PE Ratio (Price/Earnings ratio = Market Cap / Net Income)
        net_income = stock.financials.T.get("Net Income", None)
        if net_income is not None:
            yearly_data["PE_Ratio"] = yearly_data["Market_Cap"] / net_income
        else:
            yearly_data["PE_Ratio"] = None

    except Exception as e:
        print(f"⚠️ Unable to retrieve Market Cap and PE Ratio for {ticker}: {e}")

    # **Get Revenue and Net Income (longer historical data)**
    try:
        financials = stock.financials.T
        yearly_data["Revenue"] = financials.get("Total Revenue", None)
        yearly_data["Net_Income"] = financials.get("Net Income", None)
    except Exception as e:
        print(f"⚠️ Unable to retrieve financial data for {ticker}: {e}")

    # **Get ESG Score**
    try:
        sustainability = stock.sustainability
        if sustainability is not None and not sustainability.empty:
            esg_score = sustainability.loc["totalEsg", :].values[0]
        else:
            esg_score = None
        yearly_data["ESG_Score"] = esg_score
    except Exception as e:
        print(f"⚠️ Unable to retrieve ESG score for {ticker}: {e}")

    # Store the data
    all_data.append(yearly_data)

# Combine all company data into a single DataFrame
final_df = pd.concat(all_data)

# Save to a CSV file
final_df.to_csv("stock_fundamental_1993_2023.csv", index=False)
print("✅ Data successfully exported to stock_fundamental_1993_2023.csv")